## 1. Contexte

L’Etat est confronté à des contentieux dits de masse. Cela a conduit à réfléchir aux outils qui pourraient être mis en œuvre pour apporter un soutien aux services juridiques qui doivent faire face à ce genre de contentieux.
Par exemple, dans thème de la sécurité routière, les dossiers de contentieux contiennent des relevés d'information intégral (RII) qui retracent l'historique des points de l'usager qui conteste la décision de justice.

## 2. Données

Les RII sont à présent sous la forme suivante : 

![Getting Started](imgs/newrii.png){ width=50% }

Cependant, un ancien format existe encore : 

![Getting Started](imgs/vieuxrii.png){ width=50% }

De plus, les parsers pdf ou d'images conduise

## 3. Objectif : Extraction d'entités structurées

On se propose d'extraire des entités structurées contenues dans des RII, en s'appuyant sur les nouvelles techniques de traîtement du language (large language modele, LLM). En effet, bien que pour cet exemple précis, des "regex" peuvent être mises en place, on aimerait une méthode la plus généraliste possible qui se nourrie de language naturel. Ainsi, les utilisateurs métiers qui n'ont pas de compétences techniques pourront demain construire leurs propres modèles de données et les extraire de leurs documents.

--> A partir des documents présents dans le dossier `docs`, proposer une méthode d'extraction de l'entité structurée `event` qui contient les champs `date`, `heure`, `lieu`, `nature`, `points`, `numéro PV`.

## 4. Consignes

- Ne pas dépasser 4 heures pour réaliser l'objectif.
- Visez la simplicité plutôt que la complexité, la qualité plutôt que la quantité.
- Nous avons mis en place une API de LLM que vous pourrez utiliser. (Cf les exemples ci dessous).

## 5. Critères d'évaluation

- Savoir expliquer ses hypothèses, connaître les limites de son appoche.
- Qualité des réponses aux questions posées après la restitution.
- Clarté du code.

## 6. Des exemples d'utilisation de LLM :

In [2]:
!pip install langchain langchain-openai langchain-community pydantic kor load_dotenv --quiet
from dotenv import load_dotenv

load_dotenv()

import json
import os
import json
from pathlib import Path

# Import langchain
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from pydantic import BaseModel, Field
from kor import create_extraction_chain, from_pydantic

In [4]:
R2I = []
for doc in Path("docs").glob("*.txt"):
    print(doc.read_text())
    print(10*"-")
    R2I.append(doc.read_text())

# Relevé d'Infractions - Etiene Antony

  **ATTENTION : DOCUMENT CONFIDENTIEL**

*  **Date :** 

   **Heure :** 

   **Ville :** 

   12/04/2023
   15h37
   Lyon

   **Infraction :** Excess de vitesse (82 km/h dans une zone limitée à 50 km/h)

  Point perdus : **3 points**

  Numéro procès-verbal : **2023LYO0412-1537**

* **Date :** 

  **Heure :** 21h45

   **Ville :** Paris

    28/06/2023
  **Infraction :** Conduite sous l'influence de boissons alcoolisées (taux d'alcoolémie : _0.59 g/l_)

**Point perdus :** **6 points** 

Numéro procès-verbal : **2023PAR0628-2145**


* **Date :** 09/08/2023

  **Heure :** 16:12

 **Ville :** Marseille

  **Infraction :** Utilisation du téléphone portable au volant

**Point perdus :** **3 points**

Numéro procès-verbal : **2023MAR0809-1612**



* **Date :** 25/10/2023

 **Heure :** 10h08


  **Ville :** Toulouse

**Infraction :** Non-respect d'un feu rouge
* **Point perdus :** **4 points**

Numéro procès-verbal : 2023TOU1025-1008




---------------

In [2]:
llm = ChatOpenAI(temperature=0)
prompt_template = ChatPromptTemplate.from_template("Combien d'infractions dans ce document ?\n{doc}")
chain = prompt_template | llm | StrOutputParser()

/tmp/ipykernel_10118/4038688714.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0)


In [5]:
print(chain.invoke({"doc": R2I[0]}))

"Il y a **4 infractions** listées dans ce document. \n\n\nVoici les infractions et le nombre de points perdus pour chacune:\n\n* **Excess de vitesse (82 km/h dans une zone limitée à 50 km/h) :** 3 points perdus\n* **Conduite sous l'influence de boissons alcoolisées (taux d'alcoolémie : _0.59 g/l_) :** 6 points perdus\n* **Utilisation du téléphone portable au volant :** 3 points perdus\n* **Non-respect d'un feu rouge :** 4 points perdus \n"

## 7. Piste de résolution

In [9]:
class R2IInfractionEvent(BaseModel):
    date: str = Field(
        description="Date de l'infraction, sous le format jour/mois/année"
    )
    heure: str = Field(
        description="Heure de l'infraction, sous le format 'heure'h'minute'"
    )
    location: str = Field(description="Lieu de l'infraction")
    nature_infraction: str = Field(description="nature de l'infraction")
    point_loss: int = Field(
        description="Nombre de points de perdus suites à l'infraction"
    )
    pv_number: str = Field(description="Numéro du procès verbal")

schema, validator = from_pydantic(R2IInfractionEvent)
